In [ ]:
!pip install geopandas
!pip install folium
!pip install lightgbm
!pip install utm
!pip install geojson 
!pip install haversine
!pip install pyshp
!pip install pyproj 

In [ ]:
# 시각화 도구 설치
!pip install plotnine #R 시각화 
!pip install folium #지도 시각화 

# 그래프에서 한글표현을 위해 폰트를 설치합니다.
!apt-get update -qq
!apt-get install fonts-nanum* -qq

In [1]:
import matplotlib.pyplot as plt  # 그래프 그리는 용도
import matplotlib.font_manager as fm  # 폰트 관련 용도
path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()

NanumGothic Eco


In [11]:
import pandas as pd
from matplotlib import pyplot as plt 
import seaborn as sns
import numpy as np
import folium 
import geopandas as gpd
import math
from tqdm.notebook import tqdm
import utm
from shapely.geometry import Point, Polygon
from haversine import haversine
from plotnine import *
import shapefile  #the pyshp module : Should install pyshp module.
import pandas as pd
from pyproj import Proj, transform  #Should install pyproj module.

In [6]:
# shape file geojson으로 바꾸기 
import geopandas
myshpfile = geopandas.read_file('/content/drive/My Drive/Project/Seoul/기타 변수/유치원_초중고_위치데이터')
myshpfile.to_file('/content/drive/My Drive/Project/Seoul/기타 변수/유치원_초중고_위치데이터/유치원_초중고_위치데이터.geojson', driver='GeoJSON')

#geojson 읽기 
school_loc = gpd.read_file('/content/drive/My Drive/Project/Seoul/기타 변수/유치원_초중고_위치데이터/유치원_초중고_위치데이터.geojson')
school_loc

,SC_NO,SCHUL_NM,ADRES,TELNO,OLD_ZIP,TM_X,TM_Y,geometry
0,4067,금호유치원,서울특별시 성동구 금호동3가 165-1,02-2252-3179,133-805,202078.415106,549909.030647,POINT (202078.415 549909.031)
1,4068,기린유치원,서울특별시 송파구 송파동 24,02-418-1750,138-170,209509.179430,545365.823196,POINT (209509.179 545365.823)
2,3838,기청공민학교,서울특별시 서울특별시 관악구 봉천3동 37번지,02-875-4424,151-053,196241.339116,543025.738383,POINT (196241.339 543025.738)
3,4490,길음중학교,서울특별시 성북구 길음로 142,02-910-9772,136-757,201522.580709,557128.494831,POINT (201522.581 557128.495)
4,3763,까망유치원,서울특별시 성북구 장위2동 238-111,02-916-9213,136-142,204282.321509,557252.881267,POINT (204282.321 557252.881)
...,...,...,...,...,...,...,...,...
2291,4424,희망찬유치원,서울특별시 성북구 돈암1동 삼성아파트 15-3,02-926-6888,136-755,202089.637281,555525.492506,POINT (202089.637 555525.492)
2292,2471,한세사이버보안고등학교,서울특별시 마포구 아현동 457-30,02-362-0134,121-863,195758.592748,550238.821839,POINT (195758.593 550238.822)
2293,3806,한양여자대학부속유치원,서울특별시 성동구 행당1동 한양여자대학 17,02-2290-2100,133-793,204362.074026,550986.389279,POINT (204362.074 550986.389)
2294,4403,한우리유치원,서울특별시 성동구 성수2가3동 301-87,02-462-9400,133-123,204496.279589,549368.699131,POINT (204496.280 549368.699)


In [7]:
# 좌표계 확인
#  korea 2000/central belt 2010 (epsg:5186)
print('school_loc',school_loc.crs)

school_loc epsg:4326


In [8]:
school_loc.to_excel('/content/drive/My Drive/Project/Seoul/기타 변수/유치원_초중고_위치데이터/유치원_초중고_위치데이터.xlsx', index=False)

In [ ]:
# Change coordinate system
# korea 2000/central belt 2010 (epsg:5186) to wgs84(epsg:4326)
inProj = Proj(init = 'epsg:5186')
outProj= Proj(init = 'epsg:4326')
latitude = []
longitude= []
for idx,row in school_loc.iterrows():
    x,y  = row.TM_X,row.TM_Y  # korea 2000 좌표계
    nx,ny = transform(inProj,outProj,x,y)     # 새로운 좌표계    
    latitude.append(ny)
    longitude.append(nx)
school_loc['latitude'] = latitude
school_loc['longitude']= longitude

In [16]:
school_loc = school_loc.drop(['TM_X','TM_Y', 'geometry'], axis=1)
school_loc

,SC_NO,SCHUL_NM,ADRES,TELNO,OLD_ZIP,latitude,longitude
0,4067,금호유치원,서울특별시 성동구 금호동3가 165-1,02-2252-3179,133-805,37.548696,127.023520
1,4068,기린유치원,서울특별시 송파구 송파동 24,02-418-1750,138-170,37.507715,127.107550
2,3838,기청공민학교,서울특별시 서울특별시 관악구 봉천3동 37번지,02-875-4424,151-053,37.486672,126.957501
3,4490,길음중학교,서울특별시 성북구 길음로 142,02-910-9772,136-757,37.613744,127.017245
4,3763,까망유치원,서울특별시 성북구 장위2동 238-111,02-916-9213,136-142,37.614856,127.048503
...,...,...,...,...,...,...,...
2291,4424,희망찬유치원,서울특별시 성북구 돈암1동 삼성아파트 15-3,02-926-6888,136-755,37.599300,127.023663
2292,2471,한세사이버보안고등학교,서울특별시 마포구 아현동 457-30,02-362-0134,121-863,37.551660,126.952001
2293,3806,한양여자대학부속유치원,서울특별시 성동구 행당1동 한양여자대학 17,02-2290-2100,133-793,37.558395,127.049369
2294,4403,한우리유치원,서울특별시 성동구 성수2가3동 301-87,02-462-9400,133-123,37.543819,127.050878


## 행정동 Mapping

In [17]:
dong_geo = gpd.read_file('/content/drive/My Drive/Project/Seoul/서울시_행정동경계/hangjeongdong_boundary.geojson')
dong_geo

,행정동코드,시군구명,읍면동명,위도,경도,geometry
0,1111053000,종로구,사직동,37.574108,126.970144,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ..."
1,1111054000,종로구,삼청동,37.588013,126.981114,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ..."
2,1111055000,종로구,부암동,37.596699,126.962557,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ..."
3,1111056000,종로구,평창동,37.613965,126.969274,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ..."
4,1111057000,종로구,무악동,37.577741,126.958989,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ..."
...,...,...,...,...,...,...
420,1174070000,강동구,둔촌제2동,37.531514,127.147414,"MULTIPOLYGON (((127.15355 37.53400, 127.15357 ..."
421,1174057000,강동구,암사제1동,37.551855,127.134586,"MULTIPOLYGON (((127.14447 37.55477, 127.14319 ..."
422,1174061000,강동구,천호제2동,37.544691,127.122275,"MULTIPOLYGON (((127.13458 37.54712, 127.13115 ..."
423,1174068500,강동구,길동,37.539725,127.145907,"MULTIPOLYGON (((127.14857 37.54578, 127.14871 ..."


In [19]:
# creating a geometry column 
geometry = [Point(xy) for xy in zip(school_loc['longitude'], school_loc['latitude'])]
# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame 
gdf_school_loc = gpd.GeoDataFrame(school_loc, crs=crs, geometry=geometry)

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [20]:
# 행정동 포함여부 확인 
gdf_school_loc['행정동코드'] = np.zeros(len(gdf_school_loc))
gdf_school_loc['행정동'] = np.zeros(len(gdf_school_loc))

for i in gdf_school_loc.index:
    for j in dong_geo.index:
        if gdf_school_loc.geometry[i].within(dong_geo.geometry[j]):
            gdf_school_loc['행정동'][i] = dong_geo.읍면동명[j]
            gdf_school_loc['행정동코드'][i] = dong_geo.행정동코드[j]
            
gdf_school_loc['행정동코드'] = gdf_school_loc['행정동코드'].astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [21]:
gdf_school_loc

,SC_NO,SCHUL_NM,ADRES,TELNO,OLD_ZIP,latitude,longitude,geometry,행정동코드,행정동
0,4067,금호유치원,서울특별시 성동구 금호동3가 165-1,02-2252-3179,133-805,37.548696,127.023520,POINT (127.02352 37.54870),1120061500,금호2.3가동
1,4068,기린유치원,서울특별시 송파구 송파동 24,02-418-1750,138-170,37.507715,127.107550,POINT (127.10755 37.50772),1171058000,송파1동
2,3838,기청공민학교,서울특별시 서울특별시 관악구 봉천3동 37번지,02-875-4424,151-053,37.486672,126.957501,POINT (126.95750 37.48667),1162054500,청림동
3,4490,길음중학교,서울특별시 성북구 길음로 142,02-910-9772,136-757,37.613744,127.017245,POINT (127.01725 37.61374),1129066000,길음제1동
4,3763,까망유치원,서울특별시 성북구 장위2동 238-111,02-916-9213,136-142,37.614856,127.048503,POINT (127.04850 37.61486),1129077000,장위제2동
...,...,...,...,...,...,...,...,...,...,...
2291,4424,희망찬유치원,서울특별시 성북구 돈암1동 삼성아파트 15-3,02-926-6888,136-755,37.599300,127.023663,POINT (127.02366 37.59930),1129058000,돈암제1동
2292,2471,한세사이버보안고등학교,서울특별시 마포구 아현동 457-30,02-362-0134,121-863,37.551660,126.952001,POINT (126.95200 37.55166),1144055500,아현동
2293,3806,한양여자대학부속유치원,서울특별시 성동구 행당1동 한양여자대학 17,02-2290-2100,133-793,37.558395,127.049369,POINT (127.04937 37.55840),1120055000,사근동
2294,4403,한우리유치원,서울특별시 성동구 성수2가3동 301-87,02-462-9400,133-123,37.543819,127.050878,POINT (127.05088 37.54382),1120069000,성수2가제3동


In [27]:
gdf_school_loc = gdf_school_loc.sort_values(by='행정동코드').reset_index().drop('index',axis=1)

,SC_NO,행정동코드,행정동,SCHUL_NM,ADRES,TELNO,OLD_ZIP,latitude,longitude,geometry
0,3701,1111051500,청운효자동,옥인유치원,서울특별시 종로구 옥인동 18,02-735-3984,110-035,37.582177,126.970311,POINT (126.97031 37.58218)
1,4459,1111051500,청운효자동,서울맹학교,서울특별시 종로구 필운대로 103,02-737-0652,110-032,37.585135,126.968516,POINT (126.96852 37.58514)
2,3635,1111051500,청운효자동,청운중학교,서울특별시 종로구 창의문로 51,02-737-0581,110-030,37.589398,126.970755,POINT (126.97076 37.58940)
3,3666,1111051500,청운효자동,서울청운초등학교,서울특별시 종로구 자하문로 105,02-735-1924,110-030,37.585933,126.969127,POINT (126.96913 37.58593)
4,2413,1111051500,청운효자동,경기상업고등학교,서울특별시 종로구 청운동 자하문길136,02-737-0583,110-030,37.588975,126.969686,POINT (126.96969 37.58898)
...,...,...,...,...,...,...,...,...,...,...
2291,2711,1174070000,둔촌제2동,서울선린초등학교병설유치원,서울특별시 서울특별시 강동구 둔촌2동 91번지,02-477-3192,134-062,37.529489,127.145996,POINT (127.14600 37.52949)
2292,2645,1174070000,둔촌제2동,한산중학교,서울특별시 강동구 풍성로 251,02-473-6520,134-818,37.528718,127.140280,POINT (127.14028 37.52872)
2293,2243,1174070000,둔촌제2동,둔촌고등학교,서울특별시 강동구 명일로 140,02-485-9873,134-062,37.530872,127.144034,POINT (127.14403 37.53087)
2294,2704,1174070000,둔촌제2동,서울한산초등학교,서울특별시 강동구 풍성로 261,02-475-5762,134-818,37.528383,127.141358,POINT (127.14136 37.52838)


In [ ]:
gdf_school_loc = gdf_school_loc[['SC_NO','행정동코드', '행정동', 'SCHUL_NM', 'ADRES', 'TELNO', 'OLD_ZIP', 'latitude','longitude', 'geometry']]

## 유치원만 필터링

In [30]:
df_merge = gdf_school_loc.copy()
mask1 = df_merge['SCHUL_NM'].str.contains('유치원')  

df_kinder = df_merge[mask1].reset_index().drop(['index'], axis=1)
df_kinder

,SC_NO,행정동코드,행정동,SCHUL_NM,ADRES,TELNO,OLD_ZIP,latitude,longitude,geometry
0,4067,1120061500,금호2.3가동,금호유치원,서울특별시 성동구 금호동3가 165-1,02-2252-3179,133-805,37.548696,127.023520,POINT (127.02352 37.54870)
1,4068,1171058000,송파1동,기린유치원,서울특별시 송파구 송파동 24,02-418-1750,138-170,37.507715,127.107550,POINT (127.10755 37.50772)
2,3763,1129077000,장위제2동,까망유치원,서울특별시 성북구 장위2동 238-111,02-916-9213,136-142,37.614856,127.048503,POINT (127.04850 37.61486)
3,3288,1135062400,중계4동,까치유치원,서울특별시 노원구 중계4동 445,02-937-0331,139-859,37.656138,127.072967,POINT (127.07297 37.65614)
4,4069,1135061900,중계본동,꽃나라유치원,서울특별시 노원구 중계본동 18-9,02-936-2146,139-854,37.644630,127.087784,POINT (127.08778 37.64463)
...,...,...,...,...,...,...,...,...,...,...
896,4423,1135058000,월계3동,효자유치원,서울특별시 노원구 월계3동 13,02-977-8333,139-840,37.622667,127.066450,POINT (127.06645 37.62267)
897,3323,1135061900,중계본동,효천유치원,서울특별시 노원구 중계본동 366,02-930-8891,139-861,37.651801,127.079446,POINT (127.07945 37.65180)
898,4424,1129058000,돈암제1동,희망찬유치원,서울특별시 성북구 돈암1동 삼성아파트 15-3,02-926-6888,136-755,37.599300,127.023663,POINT (127.02366 37.59930)
899,3806,1120055000,사근동,한양여자대학부속유치원,서울특별시 성동구 행당1동 한양여자대학 17,02-2290-2100,133-793,37.558395,127.049369,POINT (127.04937 37.55840)


In [31]:
df_kinder.to_file('/content/drive/My Drive/Project/Seoul/기타 변수/유치원_초중고_위치데이터/유치원_위치데이터.geojson', driver='GeoJSON')

In [32]:
df_kinder.to_csv('/content/drive/My Drive/Project/Seoul/기타 변수/유치원_초중고_위치데이터/유치원_위치데이터.csv', index=False)

## 행정동 별 유치원 개수

In [50]:
df_num = df_kinder.groupby(by=['행정동','행정동코드'], as_index=False).count()[['행정동','행정동코드','SCHUL_NM']]
df_num.columns = ['행정동','행정동코드','유치원개수']

In [52]:
df_num

,행정동,행정동코드,유치원개수
0,가락1동,1171063100,1
1,가락2동,1171063200,3
2,가락본동,1171062000,2
3,가산동,1154551000,2
4,가양제1동,1150060300,3
...,...,...,...
375,회현동,1114054000,1
376,후암동,1117051000,1
377,휘경제1동,1123072000,1
378,휘경제2동,1123073000,3


In [61]:
# 결측치 0으로 채움 
df_num = dong_geo.merge(df_num, how ='outer', on='행정동코드').drop(['행정동','위도', '경도','geometry' ],axis=1).fillna(0)
df_num

,행정동코드,시군구명,읍면동명,유치원개수
0,1111053000,종로구,사직동,2.0
1,1111054000,종로구,삼청동,0.0
2,1111055000,종로구,부암동,2.0
3,1111056000,종로구,평창동,1.0
4,1111057000,종로구,무악동,1.0
...,...,...,...,...
420,1174070000,강동구,둔촌제2동,2.0
421,1174057000,강동구,암사제1동,1.0
422,1174061000,강동구,천호제2동,2.0
423,1174068500,강동구,길동,5.0


In [62]:
df_num.to_csv('/content/drive/My Drive/Project/Seoul/가공데이터/행정동별_유치원개수.csv')